In [12]:
import re
from collections import defaultdict
import pandas as pd
from tqdm import  tqdm
from threading import Thread as th
from time import time

In [39]:
def web(w):
    w = re.sub(r'www.', '', w)
    w = re.sub('https?://', '', w)
    return re.sub('/.*', '', w)

web('https://www.hello.com/world/yeah')

'hello.com'

In [79]:
def match(val1, val2, col):
    if not val1 == 'nan' and not val2 == 'nan':
        if col == 'State':
            if not val1[:3] == val2[:3]: return False
            else: return True
        if col == 'Web':
            if not val1 == val2: return False
            else: return True
        if col == 'Estb':
            if not int(val1) == int(val2): return False
            else: return True
        if col == 'Pin':
            if not int(val)
    return True

comp('har', 'del', 'State')
comp('har', 'har', 'State')
comp('Nan', 'Nan', 'State')

True

#### To get list of words in college name

In [37]:
# to_remove = ['gov\S*\s?', 'univ\S*\s?', ',*' , 'coll\S*\s?']
# pattern = re.compile('|'.join(to_remove))
def col_name(college):
    col = defaultdict()
    college = re.sub('-|_|&', '', college)
    college = re.sub('of |and ', '', college)
    
    pin = re.findall('\d{6,}', college)
    if pin: col['pin'] = int(pin[0])

    college = re.sub('\d*', '', college)
    colg = college.lower().split(',')[0]
    colg = re.sub('inst\S*\s?', 'institute ', colg)
    colg = re.sub('gov\S*\s?', 'government ', colg)
    colg = re.sub('univ\S*\s?', 'university ', colg)
    colg = re.sub('coll\S*\s?', 'college ', colg)
    colg = re.sub('eng\S*\s', 'engineering ', colg)
    colg = re.sub('tech\S*\s', 'technology ', colg)
    colg = re.sub('\s+', ' ', colg)
    colg = colg.lower().split(' ')
    col['colg'] = list(filter(None, colg))

    cities = college.lower().split(',')[1:]
    cities = [c for city in [city.split() for city in cities] for c in city]
    col['cities'] = list(filter(None, cities))
    return col

col_name('rimt- institute  of   engg. and technology , mandi - , gobindgarh 121005')

defaultdict(None,
            {'cities': ['mandi', 'gobindgarh'],
             'colg': ['rimt', 'institute', 'engineering', 'technology'],
             'pin': 121005})

In [104]:
def same(row1, row2):
    points = 0
    # if state, website or Estb unmatched, goto next row.
    if not match(row1['state'], row2['State'], 'State'): return (False, points+1)
    print('State matched')
    if row1['website'] == row2['website']: points += 2
    print('Website matched')
    
    col1 = row1['name_dict']
    col2 = row2['Name_dict']
    if col1['colg'][0] == col2['colg'][0]:
        points += 2
    points = len(set(col1['colg']).intersection(col2['colg'])) / len(set(col1['colg']).union(col2['colg']))  * 3 + points
    points = min(4, points)
    
    city1 = col1['cities'] if 'cities' in col1.keys() else []
    city2 = col1['cities'] if 'cities' in col2.keys() else []
    if not row2['City'] == 'nan' and not row1['city'] == 'nan':
        city1.append(row1['city'])
        city2.append(row2['City'])
    city1 = set(filter(None, [c[:4] for c in city1]))
    city2 = set(filter(None, [c[:4] for c in city2]))
    
    points += min(3,len(city1.intersection(city2)) * 3)
    
    if points > 4: return (True, points)
    else: return (False, points)

In [98]:
def mapper(start, end):
    global mapp, remark
    start_time = time()
    
    c360 = pd.read_csv('../all_colleges_car360.csv').astype(str)
    me = pd.read_csv('../all_colleges_mereexams.csv').astype(str)[start:end]
    c360[['State', 'City', 'Name']] = c360[['State', 'City', 'Name']].applymap(lambda x: x.lower())
    me[['state', 'city', 'name']] = me[['state', 'city', 'name']].applymap(lambda x: x.lower())
    c360.sort_values(['Name'], inplace = True)
    me.sort_values(['name'], inplace = True)
    c360['website'] = c360['website'].apply(web)
    me['website'] = me['website'].apply(web)
    print('Halfway of data loading')
    c360['Name_dict'] = c360['Name'].apply(col_name)
    me['name_dict'] = me['name'].apply(col_name)
    me['estb'] = me['year_of_establishment']
    me.drop(['year_of_establishment'], axis = 1, inplace=True)
    
    end_time = time()
    print(f'data load completed in {end_time-start_time} s')
    for index, row1 in tqdm(me.iterrows()):
        points = 0
        print(row1['name'], row1['state'], row1['website'], row1['city'], row1['estb'], sep = ' | ')
        for index2, row2 in c360.iterrows():
            if matched[index2]: continue
            (match, p)  = same(row1, row2)
            if match: 
                mapp.append([row1['name'] ,row2['Name']])
                matched[index2] = True
                break
            elif p > points:
                points = p
                name = row2['Name']
        if points > 3:
            remark.append([row1['name'] , name])

In [ ]:
t = [0] * 9
mapp = []
remark = []
matched = [False] * 18465

for i in range(8):
    t[i] = th(target = mapper, args = (i*5000, (i+1) * 5000))
t[8] = th(target = mapper, args = (40000, 42908))

for i in range(8):
    t[i].start()
t[8].start()

for i in range(8):
    t[i].join()
t[8].join()
print('script successful')

In [105]:
mapp = []
remark = []
matched = [False] * 18465
mapper(100, 110)

/home/anonymous/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Halfway of data loading




0it [00:00, ?it/s]

data load completed in 2.883502721786499 s
global business school, belagavi (pg) | karnataka | betgbs.in | belgaum | 2009.0
State matched
Website matched
State matched
Website matched
State matched
Website matched
State matched
Website matched
State matched
Website matched
State matched
Website matched




1it [00:01,  1.59s/it]

...



8it [00:14,  1.83s/it]

sri g r gandhi arts&sri y a patil comm college,  indi | karnataka | grgayapcc.org | indi | 1979.0
State matched
Website matched
State matched
Website matched
State matched
Website matched
State matched
Website matched
State matched
Website matched
State matched
Website matched




9it [00:16,  1.84s/it]

sri maliy appaiah swami edu., college,  gaddankeri | karnataka | ssvvsanghagdk.org | bagalkot | 2005.0
State matched
Website matched
State matched
Website matched
State matched
Website matched
State matched
Website matched
State matched
Website matched
State matched
Website matched




10it [00:18,  1.83s/it]



In [103]:
c360.shape

(18465, 26)

In [108]:
remark

[['global business school, belagavi (pg)',
  'sp jain school of global management, dubai']]

In [ ]:
c

In [76]:
c360.Estd.nunique()

149

In [64]:
c360 = pd.read_csv('../all_colleges_car360.csv').astype(str)
me = pd.read_csv('../all_colleges_mereexams.csv').astype(str)[100:150]
c360[['State', 'City', 'Name']] = c360[['State', 'City', 'Name']].applymap(lambda x: x.lower())
me[['state', 'city', 'name']] = me[['state', 'city', 'name']].applymap(lambda x: x.lower())
c360.sort_values(['Name'], inplace = True)
me.sort_values(['name'], inplace = True)
c360['website'] = c360['website'].apply(web)
me['website'] = me['website'].apply(web)
c360['Name_list'] = c360['Name'].apply(col_name)
me['name_list'] = me['name'].apply(col_name)
me['estb'] = me['year_of_establishment']
me.drop(['year_of_establishment'], axis = 1, inplace=True)

/home/anonymous/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
c360.loc[1, ['Name', 'City', 'State', 'website']] = me.loc[1, ['name', 'city', 'state', 'website']].values

In [8]:
print('Mereexams\n', me.columns[:15])
print('C360\n', c360.columns)

Mereexams
 Index(['cid', 'name', 'address_line1', 'address_line2', 'city', 'state',
       'pin_code', 'area', 'constructed_area', 'website',
       'year_of_establishment', 'type', 'Affiliated_to_University',
       'playground', 'auditorium'],
      dtype='object')
C360
 Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Id', 'Name',
       'known_as', 'Estd', 'State', 'City', 'Address', 'ownership',
       'Approved_by', 'Affilate_to', 'contact', 'Rank', 'website', 'score',
       'facilities', 'exams', 'stream', 'faculty', 'facebook', 'twitter',
       'youtube', 'wiki'],
      dtype='object')


In [217]:
for index, row1 in tqdm(me.iterrows()):
    points = 0
    for index, row2 in c360.iterrows():
        (match, p)  = same(row1, row2)
        if match: 
            mapp[row1['name']] = row2['Name']
            break
        elif p > points:
            points = p
            name = row2['Name']
    if points > 10:
        remark[row1['name']] = name

50it [05:23,  6.46s/it]


In [218]:
for key in mapp.keys():
    print(key, '\t', mapp[key], '\n\n') 

secab inst., of bba/bca,  college, bijapur 	 secab institute of business administration, bijapur 




In [219]:
for key in remark.keys():
    print(key, '\t', remark[key], '\n\n') 

j.h.bhalodiya women's  college-rajkot (commerce & b.b.a.) 	 shri jh bhalodiya womens college, rajkot 


jss's bharatesh college of business administration,  belgaum 	 padmaraj ariga bharatesh college of business administration, belgaum 




In [145]:
remark["indo global college of engg. abhipur"]

'indo global college of architecture, mohali'

In [136]:
me[me['name'] == 'ggs college of modern technology, kharar']

,cid,name,address_line1,address_line2,city,state,pin_code,area,constructed_area,website,...,no_of_separate_rooms_for_girls,solar_power_generation,connectivity_nkn,connectivity_nmeict,no_of_books,no_of_journals,campus_friendly,grievance_redressal_mechanism,vigilance_cell,opportunity_cell
288,C10350,"ggs college of modern technology, kharar","Chandigarh-Ludhiana Highway, Kharar",nan,"kharar, mohali",nan,140301,13.0,13877,www.ggssachdeva.com,...,1.0,False,False,False,31053.0,56.0,True,True,True,True


In [138]:
c360[c360['Name'] == 'ggs college of modern technology, mohali']

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Id,Name,known_as,Estd,State,City,Address,...,website,score,facilities,exams,stream,faculty,facebook,twitter,youtube,wiki
352,127,127,2353,2353,"ggs college of modern technology, mohali",nan,2002.0,punjab,kharar,Chandigarh-Ludhiana Highway Kharar-140301 Di...,...,http://www.ggssachdeva.com/cmt/,nan,"['Boys Hostel', 'Girls Hostel', 'Gym', 'Librar...",['JEE Main'],eng,64.0,nan,nan,nan,https://en.wikipedia.org/wiki/Guru_Gobind_Sing...


In [149]:
mapp.keys()

dict_keys(['aman bhalla institute of engg. & tech, jhakolari, pathankot', 'ggs college of modern technology, kharar', 'rimt- institute of engg. and technology, mandi gobindgarh', 'sachdeva engg. college for girls vpo mohali', 'sri sai college of engg and technology , badhani'])

In [143]:
remark.keys()

dict_keys(['aman bhalla institute of engg. & tech, jhakolari, pathankot', 'ct institute of management & technology, shahpur', 'ct institute of technology, shahpur', 'gulzar school of management, vill libra', 'guru teg bahadur khalsa institute  of engg. & technology', 'indo global college of engg. abhipur', 'sachdeva engg. college for girls vpo mohali', 'sri sai college of engg and technology , badhani', 'swami parmanand engg. college'])

In [99]:
me[me['name'] == 'ct institute of management & technology, shahpur']

,cid,name,address_line1,address_line2,city,state,pin_code,area,constructed_area,website,...,no_of_separate_rooms_for_girls,solar_power_generation,connectivity_nkn,connectivity_nmeict,no_of_books,no_of_journals,campus_friendly,grievance_redressal_mechanism,vigilance_cell,opportunity_cell
335,C10416,"ct institute of management & technology, shahpur","VILLAGE SHAHPUR, P.O. UDOPUR, VIA JAMSHER KHAS...","VILLAGE SHAHPUR, P.O. UDOPUR, VIA JAMSHER KHAS...",jalandhar,nan,144020,3.0,10494,http://www.ctgroup.in,...,1.0,False,True,True,nan,nan,True,True,True,True


In [154]:
c360[c360['Name'] == 'yogi vemana university, kadapa']

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Id,Name,known_as,Estd,State,City,Address,...,website,score,facilities,exams,stream,faculty,facebook,twitter,youtube,wiki
32,32,11,140,140,"yogi vemana university, kadapa",nan,2006.0,andhra pradesh,kopparthi,Kadapa – 516 003,...,http://www.yogivemanauniversity.ac.in/,nan,"['Boys Hostel', 'Girls Hostel', 'Gym', 'Librar...",[' IPU CET BJMC '],media,108.0,nan,nan,nan,https://en.wikipedia.org/wiki/Yogi_Vemana_Univ...
112,112,11,140,140,"yogi vemana university, kadapa",nan,2006.0,andhra pradesh,kopparthi,Kadapa – 516 003,...,http://www.yogivemanauniversity.ac.in/,nan,"['Boys Hostel', 'Girls Hostel', 'Gym', 'Librar...",[' IPU CET BJMC '],media,108.0,nan,nan,nan,https://en.wikipedia.org/wiki/Yogi_Vemana_Univ...


In [14]:
    c360 = pd.read_csv('../all_colleges_car360.csv').astype(str)
    me = pd.read_csv('../all_colleges_mereexams.csv').astype(str)[0:1000]
    c360[['State', 'City', 'Name']] = c360[['State', 'City', 'Name']].applymap(lower)
    me[['state', 'city', 'name']] = me[['state', 'city', 'name']].applymap(lower)
    c360.sort_values(['Name'], inplace = True)
    me.sort_values(['name'], inplace = True)
    c360['website'] = c360['website'].apply(web)
    me['website'] = me['website'].apply(web)
    print('Halfway of data loading')
    c360['Name'] = c360['Name'].apply(col_name)
    me['name'] = me['name'].apply(col_name)
    me['estb'] = me['year_of_establishment']
    me.drop(['year_of_establishment'], axis = 1, inplace=True)


/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Halfway of data loading


In [32]:
for id, clg in remark:
    print(me[me['cid'] == id]['name'].values, clg, end = '\n\n')

[list(["b.v.v.sangha's", 'basaveshwar', 'commercecollege', 'bagalkot'])] ['basaveshwar', 'commerce', 'college', 'bagalkot']

[list(['bldeas', 'comm', 'bhs', 'arts', 'tgp', 'sci', 'college', 'jamakhandi'])] ["bldea's", 'commerce', 'bhs', 'arts', 'tgp', 'science', 'college', 'jamkhandi']

[list(['bldeas', 'j', 's', 's', 'college', 'edu.', 'bijapur'])] ["bldea's", 'vp', 'dr', 'pg', 'halakatti', 'college', 'engineering', 'technology', 'bijapur']

[list(['first', 'grade', 'college', 'muddebihal'])] ['first', 'grade', 'college', 'ankola']

[list(['j.h.bhalodiya', "women's", 'collegerajkot', '(commerce', 'b.b.a.)'])] ['shri', 'jh', 'bhalodiya', 'womens', 'college', 'rajkot']

[list(["jss's", 'bharatesh', 'college', 'business', 'administration', 'belgaum'])] ['padmaraj', 'ariga', 'bharatesh', 'college', 'business', 'administration', 'belgaum']

[list(['secab', 'bba/bca', 'college', 'bijapur'])] ['secab', 'business', 'administration', 'bijapur']

[list(['shree', 'a.v.j.', 'oza', 'b.ed.', 'colle